# PA004 - HEALTH INSURANCE CROSS-SELL

## 0- IMPORTS

In [1]:
import os

import pandas          as pd
import numpy           as np
import pandas.io.sql   as psql
import psycopg2        as pg
import sweetviz        as sv

/home/reng/anaconda3/envs/health_insurance/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2 - DATAS

#### Schema Query

In [2]:
# VERIFICANDO SCHEMAS DO BANCO DE DADOS
query_schema = """
    SELECT nspname
    FROM pg_catalog.pg_namespace
"""
cur.execute( query_schema )

record = cur.fetchall()
print(record)

NameError: name 'cur' is not defined

#### Table Query

In [ ]:
# VERIFICANDO TABELAS BANCO DE DADOS
query_table = """
    SELECT tablename
    FROM pg_tables
    WHERE schemaname='pa004'
"""

cur.execute( query_table )

record = cur.fetchall()
print(record)

#### Colecting Datas from DataBase using Pandas

In [ ]:
query_all_datas = """
    SELECT *
    FROM pa004.users u
    INNER JOIN pa004.vehicle v ON ( u.id = v.id )
    INNER JOIN pa004.insurance i ON (u.id = i.id )
"""
df_raw = pd.read_sql( query_all_datas, conn )
cur.close()
conn.close()

### 2.1 - Os dados foram coletados, limpos e salvos localmente, portando as credenciais de acesso ao banco de dados não estou disponíveis neste notebook.

In [ ]:
# Salvando os dados coletados em csv na minha máquina
os.makedirs('/home/reng/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/data', exist_ok=True) #Criando pasta
df2.to_csv('/home/reng/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/data/data.csv')# Salvando os dados coletados em csv na minha máquina

In [3]:
# Carregando os dados salvos
df = pd.read_csv('/home/reng/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/data/data.csv')

### 2.2 - Data Description

In [4]:
df1 = df.copy()

### 2.3 - Size of Dataset

In [5]:
df1.shape

(381109, 13)

### 2.4 - Check Data Types

In [6]:
df1.dtypes

Unnamed: 0                int64
id                        int64
gender                   object
age                       int64
region_code             float64
policy_sales_channel    float64
driving_license           int64
vehicle_age              object
vehicle_damage           object
previously_insured        int64
annual_premium          float64
vintage                   int64
response                  int64
dtype: object

### 2.5 - Check NA Datas

In [7]:
df1.isna().sum()

Unnamed: 0              0
id                      0
gender                  0
age                     0
region_code             0
policy_sales_channel    0
driving_license         0
vehicle_age             0
vehicle_damage          0
previously_insured      0
annual_premium          0
vintage                 0
response                0
dtype: int64

In [8]:
df1.shape

(381109, 13)

### 2.6 - Removing Duplicated Columns

In [9]:
df1.head()

,Unnamed: 0,id,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,0,7,Male,23,11.0,152.0,1,< 1 Year,Yes,0,23367.0,249,0
1,1,13,Female,41,15.0,14.0,1,1-2 Year,No,1,31409.0,221,0
2,2,18,Female,25,35.0,152.0,1,< 1 Year,No,1,46622.0,299,0
3,3,31,Female,26,8.0,160.0,1,< 1 Year,No,0,2630.0,136,0
4,4,39,Male,45,8.0,124.0,1,1-2 Year,Yes,0,42297.0,264,0


In [ ]:
df1 = df1.loc[ :,~df1.columns.duplicated() ]

In [ ]:
df1.shape

### 2.7 - Descriptive Statisctical

In [ ]:
df1['vehicle_age'].unique()

In [ ]:
num_attributes = df1.select_dtypes( include=[ 'int32', 'int64', 'float64' ] )
cat_attributes = df1.select_dtypes( exclude=[ 'object'] )

In [ ]:
num_attributes

In [ ]:
# Central Tendency - mean, median
ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame( num_attributes.apply( np.median ) ).T

# Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T
d2 = pd.DataFrame( num_attributes.apply( min ) ).T
d3 = pd.DataFrame( num_attributes.apply( max ) ).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T

# Concatenar
m = pd.concat( [ d2, d3, d4, ct1, ct2, d1, d5, d6 ] ).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

## 3 - FEATURE ENGINEERING

In [ ]:
df2 = df1.copy()